In [1]:
import os
import pandas as pd
import re
import math
import duckdb
import numpy as np
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('my_database.duckdb')

In [2]:
# Read the CSV file line by line
file_path = 'decoded.csv'
lines = []

with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize dictionaries to store lines for each transaction type
transaction_dict = {}

# Function to parse fields without combining text between single quotes
def parse_line(line):
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    return fields

# Iterate over each line to categorize it based on the 6th field
for line in lines:
    fields = parse_line(line.strip())
    if len(fields) > 5:  # Ensure there are at least 6 fields after cleaning
        transaction_type = fields[5]
        if transaction_type not in transaction_dict:
            transaction_dict[transaction_type] = []
        transaction_dict[transaction_type].append(fields)

# Create separate DataFrames for each transaction type
dataframes = {}
for transaction_type, data in transaction_dict.items():
    # Determine the number of columns for this transaction type
    num_columns = max(len(fields) for fields in data)
    # Pad the data with empty strings to ensure all rows have the same number of columns
    padded_data = [fields + [''] * (num_columns - len(fields)) for fields in data]
    # Create DataFrame
    df = pd.DataFrame(padded_data, columns=[f'Field_{i+1}' for i in range(num_columns)])
    dataframes[transaction_type] = df

# Print out the names of the created DataFrames
for transaction_type in dataframes.keys():
    print(f'Created DataFrame: {transaction_type}')


Created DataFrame: Swap
Created DataFrame: PoolRegistered
Created DataFrame: TokensRegistered
Created DataFrame: PoolBalanceChanged
Created DataFrame: InternalBalanceChanged
Created DataFrame: RelayerApprovalChanged
Created DataFrame: ExternalBalanceTransfer
Created DataFrame: FlashLoan
Created DataFrame: PoolBalanceManaged
Created DataFrame: AuthorizerChanged


In [3]:
# Function to combine fields after Field_6 and before '0x'
def combine_fields(df):
    combined_data = []
    for index, row in df.iterrows():
        combined_row = row[:6].tolist()  # Get the first six fields
        remaining_fields = row[6:].tolist()  # Get the remaining fields
        combined_field = []
        for field in remaining_fields:
            if re.match(r'^0x', field):  # Check for '0x' pattern
                break
            combined_field.append(field)
        combined_row.append(' '.join(combined_field))
        combined_data.append(combined_row + remaining_fields[len(combined_field):])
    return pd.DataFrame(combined_data, columns=[f'Field_{i+1}' for i in range(len(combined_data[0]))])

dataframes['Swap'] = combine_fields(dataframes['Swap'])
dataframes['PoolRegistered'] = combine_fields(dataframes['PoolRegistered'])
dataframes['PoolBalanceChanged'] = combine_fields(dataframes['PoolBalanceChanged'])
dataframes['PoolBalanceManaged'] = combine_fields(dataframes['PoolBalanceManaged'])

In [4]:
Swap = dataframes['Swap']
PoolRegistered = dataframes['PoolRegistered']
PoolBalanceChanged = dataframes['PoolBalanceChanged']
PoolBalanceManaged = dataframes['PoolBalanceManaged']
ExternalBalanceTransfer = dataframes['ExternalBalanceTransfer']
FlashLoan = dataframes['FlashLoan']
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16,Field_17
0,15718738,167,275,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\xae7\xd5J\xe4w&\x8b\x99\x97\xd4\x16\x1b\x96\xb8 \x07U\x93\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x037',0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x02d60b84491589974263d922D9cC7a3152618Ef6,-2784083623838693761194534,2784083623838693761194534,None,None,None,None,None,
1,15718738,167,276,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\xae7\xd5J\xe4w&\x8b\x99\x97\xd4\x16\x1b\x96\xb8 \x07U\x93\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x037',0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x02d60b84491589974263d922D9cC7a3152618Ef6,0,-2784083623838693761194534,None,None,None,None,None,
2,15718738,167,286,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\xae7\xd5J\xe4w&\x8b\x99\x97\xd4\x16\x1b\x96\xb8 \x07U\x93\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x037',0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x6B175474E89094C44Da98b954EedeAC495271d0F,0,2996578729376462809176697,None,None,None,None,None,
3,15718738,167,288,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\xae7\xd5J\xe4w&\x8b\x99\x97\xd4\x16\x1b\x96\xb8 \x07U\x93\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x037',0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x6B175474E89094C44Da98b954EedeAC495271d0F,2996578729376462809176697,-2996578729376462809176697,None,None,None,None,None,
4,15768529,155,287,0x592b8bf9ee2d8c8aa1d284a4ef86395ce63a77396083a924db1898af226e2c45,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x82i\x8a\xec\xc9\xe2\x8e\x9b\xb2v\x08\xbdR\xcfW\xf7\x04\xbd\x1b\x83\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x036',0x38C6Cb265caaf011A390BA6FF21d8b51388973Cc,0xd093fA4Fb80D09bB30817FDcd442d4d02eD3E5de,-8853187579584,8853187579584,None,None,None,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,18979523,54,66,0xbdeb1d9d676bad54d707234e85dab3695b1c89d2d227a69fbdb0b1bfdebfb912,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'd\x1aRu\x1c\x1a \xd1c<\xcb\xd1\xbf\xec!>\xc08\xc4U\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x05\x97',0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0,-3444819,None,None,None,None,None,None
1112,18979639,125,210,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'd\x1aRu\x1c\x1a \xd1c<\xcb\xd1\xbf\xec!>\xc08\xc4U\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x05\x97',0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,-415906692235215384657,415906692235215384657,None,None,None,None,None,None
1113,18979639,125,212,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'd\x1aRu\x1c\x1a \xd1c<\xcb\xd1\xbf\xec!>\xc08\xc4U\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x05\x97',0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,-2264502695826,2264502695826,None,None,None,None,None,None
1114,18979639,125,213,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'd\x1aRu\x1c\x1a \xd1c<\xcb\xd1\xbf\xec!>\xc08\xc4U\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x05\x97',0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,0,-415906692235215384657,None,None,None,None,None,None


In [5]:
# Function to add double quotes around the value
def byte_to_hex(byte_string):
    # Convert the string representation of the byte string to an actual byte string
    byte_string = eval(byte_string)
    # Convert the byte string to a hexadecimal string
    return byte_string.hex()

# Apply the function to the Field_7 column
PoolBalanceChanged['Field_7'] = PoolBalanceChanged['Field_7'].apply(byte_to_hex)
PoolBalanceManaged['Field_7'] = PoolBalanceManaged['Field_7'].apply(byte_to_hex)
Swap['Field_7'] = Swap['Field_7'].apply(byte_to_hex)

In [6]:
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16,Field_17
0,15718738,167,275,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,ae37d54ae477268b9997d4161b96b8200755935c000000000000000000000337,0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x02d60b84491589974263d922D9cC7a3152618Ef6,-2784083623838693761194534,2784083623838693761194534,None,None,None,None,None,
1,15718738,167,276,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,ae37d54ae477268b9997d4161b96b8200755935c000000000000000000000337,0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x02d60b84491589974263d922D9cC7a3152618Ef6,0,-2784083623838693761194534,None,None,None,None,None,
2,15718738,167,286,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,ae37d54ae477268b9997d4161b96b8200755935c000000000000000000000337,0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x6B175474E89094C44Da98b954EedeAC495271d0F,0,2996578729376462809176697,None,None,None,None,None,
3,15718738,167,288,0xd0bda256888da55c16cdc987e1e61595be30b27edfb90e748fca58edd759156a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,ae37d54ae477268b9997d4161b96b8200755935c000000000000000000000337,0x1E84bf59f48Ddb2Bb2AC70bD18d153381906128C,0x6B175474E89094C44Da98b954EedeAC495271d0F,2996578729376462809176697,-2996578729376462809176697,None,None,None,None,None,
4,15768529,155,287,0x592b8bf9ee2d8c8aa1d284a4ef86395ce63a77396083a924db1898af226e2c45,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,82698aecc9e28e9bb27608bd52cf57f704bd1b83000000000000000000000336,0x38C6Cb265caaf011A390BA6FF21d8b51388973Cc,0xd093fA4Fb80D09bB30817FDcd442d4d02eD3E5de,-8853187579584,8853187579584,None,None,None,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,18979523,54,66,0xbdeb1d9d676bad54d707234e85dab3695b1c89d2d227a69fbdb0b1bfdebfb912,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,641a52751c1a20d1633ccbd1bfec213ec038c455000200000000000000000597,0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0,-3444819,None,None,None,None,None,None
1112,18979639,125,210,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,641a52751c1a20d1633ccbd1bfec213ec038c455000200000000000000000597,0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,-415906692235215384657,415906692235215384657,None,None,None,None,None,None
1113,18979639,125,212,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,641a52751c1a20d1633ccbd1bfec213ec038c455000200000000000000000597,0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,-2264502695826,2264502695826,None,None,None,None,None,None
1114,18979639,125,213,0xe6724cfd8656c3ff5557980768498ee03b21c173a9ba8b530cab8966b27b612e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,641a52751c1a20d1633ccbd1bfec213ec038c455000200000000000000000597,0xBEbB92Ed09688e64DC38c240B600D0B1d504EE56,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,0,-415906692235215384657,None,None,None,None,None,None


In [7]:
PoolBalanceChanged[['Field_9', 'Field_10', 'Field_11', 'Field_12', 'Field_13', 'Field_14']].to_csv('PoolBalanceChanged.txt')

In [8]:
import pandas as pd
import re

# Read the text file
file_path = 'PoolBalanceChanged.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Function to replace commas with dashes within square brackets
def replace_commas(text):
    return re.sub(r'\[([^\]]+)\]', lambda m: '[' + m.group(1).replace(',', '/') + ']', text)

# Apply the function to each line
modified_lines = [replace_commas(line) for line in lines]

# Save the modified lines back to a new text file (optional)
output_path = 'PoolBalanceChanged_modified.txt'
with open(output_path, 'w') as file:
    file.writelines(modified_lines)

# Create the pandas dataframe
PoolBalanceChanged_lists = pd.read_csv('PoolBalanceChanged_modified.txt')
PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2'/'0xba100000625a3754423978a60c9317c58a424e3D'],[37500000000000000/3000000000000000000],[0/0],NaN,NaN,NaN
1,1,['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'/'0xdAC17F958D2ee523a2206206994597C13D831ec7'],[60000000000000000/250000000],[0/0],NaN,NaN,NaN
2,2,['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'/'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],[0/43299378420984092],[0/0],NaN,NaN,NaN
3,3,['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'/'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],[-126324/-21775305304298319],[0/0],NaN,NaN,NaN
4,4,['0xB4EFd85c19999D84251304bDA99E90B92300Bd93'/'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],[9835541847739915246/24450339974267058],[0/0],NaN,NaN,NaN
...,...,...,...,...,...,...,...
168800,168800,['0x83F20F44975D03b1b09e64809B757c47f942BEeA'/'0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A'],[-596755585815051015831/-730273629479456603273],[0/0],NaN,NaN,NaN
168801,168801,['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0'/'0xBe9895146f7AF43049ca1c1AE358B0541Ea49704'],[-20706638154657043/-458996031245460252],[0/0],NaN,NaN,NaN
168802,168802,['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893'/'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],[-6365511991583158775849/-89963497755758211],[0/0],NaN,NaN,NaN
168803,168803,['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'/'0xcE246eEa10988C495B4A90a905Ee9237a0f91543'],[90495470289569876/9875000000000000000000],[0/1252291690311437084314],NaN,NaN,NaN


In [9]:

# Function to convert the string to a list of strings
def convert_to_list(cell_value):
    if isinstance(cell_value, str):
        # Remove brackets
        cell_value = cell_value.strip('[]')
        # Split by '/'
        split_values = cell_value.split('/')
        # Convert to list of strings
        return [str(value) for value in split_values]
    else:
        return cell_value

# Apply the function to all columns
for column in ['Field_9', 'Field_10', 'Field_11']:
    PoolBalanceChanged_lists[column] = PoolBalanceChanged_lists[column].apply(convert_to_list)

PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,"['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0xba100000625a3754423978a60c9317c58a424e3D']","[37500000000000000, 3000000000000000000]","[0, 0]",NaN,NaN,NaN
1,1,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7']","[60000000000000000, 250000000]","[0, 0]",NaN,NaN,NaN
2,2,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, 43299378420984092]","[0, 0]",NaN,NaN,NaN
3,3,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-126324, -21775305304298319]","[0, 0]",NaN,NaN,NaN
4,4,"['0xB4EFd85c19999D84251304bDA99E90B92300Bd93', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[9835541847739915246, 24450339974267058]","[0, 0]",NaN,NaN,NaN
...,...,...,...,...,...,...,...
168800,168800,"['0x83F20F44975D03b1b09e64809B757c47f942BEeA', '0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A']","[-596755585815051015831, -730273629479456603273]","[0, 0]",NaN,NaN,NaN
168801,168801,"['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704']","[-20706638154657043, -458996031245460252]","[0, 0]",NaN,NaN,NaN
168802,168802,"['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-6365511991583158775849, -89963497755758211]","[0, 0]",NaN,NaN,NaN
168803,168803,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xcE246eEa10988C495B4A90a905Ee9237a0f91543']","[90495470289569876, 9875000000000000000000]","[0, 1252291690311437084314]",NaN,NaN,NaN


In [10]:
PoolBalanceChanged['Field_9'] = PoolBalanceChanged_lists['Field_9']
PoolBalanceChanged['Field_10'] = PoolBalanceChanged_lists['Field_10']
PoolBalanceChanged['Field_11'] = PoolBalanceChanged_lists['Field_11']
PoolBalanceChanged = PoolBalanceChanged[[f'Field_{i}' for i in range(1, 12)]]
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,12377323,17,33,0x202e139f88f9c4d436d2f71e837ce5be3297a0cc652a97377b85eeaa11631336,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,03cd191f589d12b0582a99808cf19851e468e6b500020000000000000000002b,0x089775364EA24957dA94348cf740375826Ed3330,"['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0xba100000625a3754423978a60c9317c58a424e3D']","[37500000000000000, 3000000000000000000]","[0, 0]"
1,12377580,58,120,0x139f925f2b47a67461a62881a45cde7096d2691e51a970105ef6d1eea6eae164,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,14bf727f67aa294ec36347bd95aba1a2c136fe7a000200000000000000000020,0x089775364EA24957dA94348cf740375826Ed3330,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7']","[60000000000000000, 250000000]","[0, 0]"
2,12377610,264,292,0xac68b42bc969c39e22348aabb841a4bc8e8f0bc77ebda186181f54bf7308cb7f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, 43299378420984092]","[0, 0]"
3,12377634,19,25,0x217a997e4b3ecb30ab0037c59d23a306bff93ef067b2129b9c1e69ad029f3684,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-126324, -21775305304298319]","[0, 0]"
4,12378615,512,140,0x7c41ee88c46f11b3739e65499e7e60328d65e8bdddf8fea0ab9d13b9778f3507,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,16faf9f73748013155b7bc116a3008b57332d1e600020000000000000000001c,0xa54a561dcBc3D7d344D4150e3465993fFB1DD7B6,"['0xB4EFd85c19999D84251304bDA99E90B92300Bd93', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[9835541847739915246, 24450339974267058]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
168800,20007031,57,200,0xcc9c22c5e5c0f43934b6641b3313c8b927f3d4dc7234850cd32d0e2f28e1e102,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,1cce5169bde03f3d5ad0206f6bd057953539dae600020000000000000000062b,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x83F20F44975D03b1b09e64809B757c47f942BEeA', '0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A']","[-596755585815051015831, -730273629479456603273]","[0, 0]"
168801,20007039,39,132,0x18f13c3a35e249034b74fa314850999cb6d1e089d11743fd4ecac50b351806b0,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,f7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704']","[-20706638154657043, -458996031245460252]","[0, 0]"
168802,20007213,46,81,0xeea539d2a744427c1b10a116451b7db62a1de6daa448ab63595603c4ab0450a6,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cf7b51ce5755513d4be016b0e28d6edeffa1d52a000200000000000000000617,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-6365511991583158775849, -89963497755758211]","[0, 0]"
168803,20007271,160,427,0x328a0ba800c03851bccab0af1c1f76c32e2457b1a9f7cdd38826c3016b21c57c,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,577a7f7ee659aa14dc16fd384b3f8078e23f1920000200000000000000000633,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xcE246eEa10988C495B4A90a905Ee9237a0f91543']","[90495470289569876, 9875000000000000000000]","[0, 1252291690311437084314]"


In [11]:
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,12377323,17,33,0x202e139f88f9c4d436d2f71e837ce5be3297a0cc652a97377b85eeaa11631336,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,03cd191f589d12b0582a99808cf19851e468e6b500020000000000000000002b,0x089775364EA24957dA94348cf740375826Ed3330,"['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0xba100000625a3754423978a60c9317c58a424e3D']","[37500000000000000, 3000000000000000000]","[0, 0]"
1,12377580,58,120,0x139f925f2b47a67461a62881a45cde7096d2691e51a970105ef6d1eea6eae164,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,14bf727f67aa294ec36347bd95aba1a2c136fe7a000200000000000000000020,0x089775364EA24957dA94348cf740375826Ed3330,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7']","[60000000000000000, 250000000]","[0, 0]"
2,12377610,264,292,0xac68b42bc969c39e22348aabb841a4bc8e8f0bc77ebda186181f54bf7308cb7f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, 43299378420984092]","[0, 0]"
3,12377634,19,25,0x217a997e4b3ecb30ab0037c59d23a306bff93ef067b2129b9c1e69ad029f3684,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-126324, -21775305304298319]","[0, 0]"
4,12378615,512,140,0x7c41ee88c46f11b3739e65499e7e60328d65e8bdddf8fea0ab9d13b9778f3507,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,16faf9f73748013155b7bc116a3008b57332d1e600020000000000000000001c,0xa54a561dcBc3D7d344D4150e3465993fFB1DD7B6,"['0xB4EFd85c19999D84251304bDA99E90B92300Bd93', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[9835541847739915246, 24450339974267058]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
168800,20007031,57,200,0xcc9c22c5e5c0f43934b6641b3313c8b927f3d4dc7234850cd32d0e2f28e1e102,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,1cce5169bde03f3d5ad0206f6bd057953539dae600020000000000000000062b,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x83F20F44975D03b1b09e64809B757c47f942BEeA', '0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A']","[-596755585815051015831, -730273629479456603273]","[0, 0]"
168801,20007039,39,132,0x18f13c3a35e249034b74fa314850999cb6d1e089d11743fd4ecac50b351806b0,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,f7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704']","[-20706638154657043, -458996031245460252]","[0, 0]"
168802,20007213,46,81,0xeea539d2a744427c1b10a116451b7db62a1de6daa448ab63595603c4ab0450a6,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cf7b51ce5755513d4be016b0e28d6edeffa1d52a000200000000000000000617,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-6365511991583158775849, -89963497755758211]","[0, 0]"
168803,20007271,160,427,0x328a0ba800c03851bccab0af1c1f76c32e2457b1a9f7cdd38826c3016b21c57c,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,577a7f7ee659aa14dc16fd384b3f8078e23f1920000200000000000000000633,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xcE246eEa10988C495B4A90a905Ee9237a0f91543']","[90495470289569876, 9875000000000000000000]","[0, 1252291690311437084314]"


In [12]:
# Select and rename specific columns
selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'tokens', 'Field_10': 'delta', 'Field_11': 'protocol_fee'}
PoolBalanceChanged = PoolBalanceChanged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'token', 'Field_10': 'cash_delta'}
PoolBalanceManaged = PoolBalanceManaged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'token_in', 'Field_9': 'token_out', 'Field_10': 'amount_in', 'Field_11': 'amount_out'}
Swap = Swap[list(selected_columns.keys())].rename(columns=selected_columns)

PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,12377323,33,0x202e139f88f9c4d436d2f71e837ce5be3297a0cc652a97377b85eeaa11631336,PoolBalanceChanged,03cd191f589d12b0582a99808cf19851e468e6b500020000000000000000002b,0x089775364EA24957dA94348cf740375826Ed3330,"['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0xba100000625a3754423978a60c9317c58a424e3D']","[37500000000000000, 3000000000000000000]","[0, 0]"
1,12377580,120,0x139f925f2b47a67461a62881a45cde7096d2691e51a970105ef6d1eea6eae164,PoolBalanceChanged,14bf727f67aa294ec36347bd95aba1a2c136fe7a000200000000000000000020,0x089775364EA24957dA94348cf740375826Ed3330,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7']","[60000000000000000, 250000000]","[0, 0]"
2,12377610,292,0xac68b42bc969c39e22348aabb841a4bc8e8f0bc77ebda186181f54bf7308cb7f,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, 43299378420984092]","[0, 0]"
3,12377634,25,0x217a997e4b3ecb30ab0037c59d23a306bff93ef067b2129b9c1e69ad029f3684,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-126324, -21775305304298319]","[0, 0]"
4,12378615,140,0x7c41ee88c46f11b3739e65499e7e60328d65e8bdddf8fea0ab9d13b9778f3507,PoolBalanceChanged,16faf9f73748013155b7bc116a3008b57332d1e600020000000000000000001c,0xa54a561dcBc3D7d344D4150e3465993fFB1DD7B6,"['0xB4EFd85c19999D84251304bDA99E90B92300Bd93', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[9835541847739915246, 24450339974267058]","[0, 0]"
...,...,...,...,...,...,...,...,...,...
168800,20007031,200,0xcc9c22c5e5c0f43934b6641b3313c8b927f3d4dc7234850cd32d0e2f28e1e102,PoolBalanceChanged,1cce5169bde03f3d5ad0206f6bd057953539dae600020000000000000000062b,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x83F20F44975D03b1b09e64809B757c47f942BEeA', '0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A']","[-596755585815051015831, -730273629479456603273]","[0, 0]"
168801,20007039,132,0x18f13c3a35e249034b74fa314850999cb6d1e089d11743fd4ecac50b351806b0,PoolBalanceChanged,f7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704']","[-20706638154657043, -458996031245460252]","[0, 0]"
168802,20007213,81,0xeea539d2a744427c1b10a116451b7db62a1de6daa448ab63595603c4ab0450a6,PoolBalanceChanged,cf7b51ce5755513d4be016b0e28d6edeffa1d52a000200000000000000000617,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-6365511991583158775849, -89963497755758211]","[0, 0]"
168803,20007271,427,0x328a0ba800c03851bccab0af1c1f76c32e2457b1a9f7cdd38826c3016b21c57c,PoolBalanceChanged,577a7f7ee659aa14dc16fd384b3f8078e23f1920000200000000000000000633,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xcE246eEa10988C495B4A90a905Ee9237a0f91543']","[90495470289569876, 9875000000000000000000]","[0, 1252291690311437084314]"


In [13]:
PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,12377323,33,0x202e139f88f9c4d436d2f71e837ce5be3297a0cc652a97377b85eeaa11631336,PoolBalanceChanged,03cd191f589d12b0582a99808cf19851e468e6b500020000000000000000002b,0x089775364EA24957dA94348cf740375826Ed3330,"['0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', '0xba100000625a3754423978a60c9317c58a424e3D']","[37500000000000000, 3000000000000000000]","[0, 0]"
1,12377580,120,0x139f925f2b47a67461a62881a45cde7096d2691e51a970105ef6d1eea6eae164,PoolBalanceChanged,14bf727f67aa294ec36347bd95aba1a2c136fe7a000200000000000000000020,0x089775364EA24957dA94348cf740375826Ed3330,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7']","[60000000000000000, 250000000]","[0, 0]"
2,12377610,292,0xac68b42bc969c39e22348aabb841a4bc8e8f0bc77ebda186181f54bf7308cb7f,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, 43299378420984092]","[0, 0]"
3,12377634,25,0x217a997e4b3ecb30ab0037c59d23a306bff93ef067b2129b9c1e69ad029f3684,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000200000000000000000003,0x22Cd4027b8Af39de8eB8809588a60b8EF067eF76,"['0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-126324, -21775305304298319]","[0, 0]"
4,12378615,140,0x7c41ee88c46f11b3739e65499e7e60328d65e8bdddf8fea0ab9d13b9778f3507,PoolBalanceChanged,16faf9f73748013155b7bc116a3008b57332d1e600020000000000000000001c,0xa54a561dcBc3D7d344D4150e3465993fFB1DD7B6,"['0xB4EFd85c19999D84251304bDA99E90B92300Bd93', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[9835541847739915246, 24450339974267058]","[0, 0]"
...,...,...,...,...,...,...,...,...,...
168800,20007031,200,0xcc9c22c5e5c0f43934b6641b3313c8b927f3d4dc7234850cd32d0e2f28e1e102,PoolBalanceChanged,1cce5169bde03f3d5ad0206f6bd057953539dae600020000000000000000062b,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x83F20F44975D03b1b09e64809B757c47f942BEeA', '0xe07F9D810a48ab5c3c914BA3cA53AF14E4491e8A']","[-596755585815051015831, -730273629479456603273]","[0, 0]"
168801,20007039,132,0x18f13c3a35e249034b74fa314850999cb6d1e089d11743fd4ecac50b351806b0,PoolBalanceChanged,f7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704']","[-20706638154657043, -458996031245460252]","[0, 0]"
168802,20007213,81,0xeea539d2a744427c1b10a116451b7db62a1de6daa448ab63595603c4ab0450a6,PoolBalanceChanged,cf7b51ce5755513d4be016b0e28d6edeffa1d52a000200000000000000000617,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0x137dDB47Ee24EaA998a535Ab00378d6BFa84F893', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[-6365511991583158775849, -89963497755758211]","[0, 0]"
168803,20007271,427,0x328a0ba800c03851bccab0af1c1f76c32e2457b1a9f7cdd38826c3016b21c57c,PoolBalanceChanged,577a7f7ee659aa14dc16fd384b3f8078e23f1920000200000000000000000633,0xb9Ea80d79bD32159AD9B34aD2C83d6C5f9ee3968,"['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xcE246eEa10988C495B4A90a905Ee9237a0f91543']","[90495470289569876, 9875000000000000000000]","[0, 1252291690311437084314]"


In [14]:
con.execute('CREATE TABLE mainnet_swap AS SELECT * FROM Swap').fetchdf()
con.execute('CREATE TABLE mainnet_poolregistered AS SELECT * FROM PoolRegistered').fetchdf()
con.execute('CREATE TABLE mainnet_poolbalancechanged AS SELECT * FROM PoolBalanceChanged').fetchdf()
con.execute('CREATE TABLE mainnet_externalbalancetransfer AS SELECT * FROM ExternalBalanceTransfer').fetchdf()
con.execute('CREATE TABLE mainnet_flashloan AS SELECT * FROM FlashLoan').fetchdf()
con.execute('CREATE TABLE mainnet_poolbalancemanaged AS SELECT * FROM PoolBalanceManaged').fetchdf()

# Commit the changes
con.commit()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
PoolBalanceChanged[PoolBalanceChanged['pool_id']== '5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014']  

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
6,12379028,46,0x635808ef022fefbd2a3a8469dfadfd90382d8fa6c828197a788d8d51d2eb2010,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x1BC4230d996286ffD1dD4DC649AAd47Fa08cadc4,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[7374088169332141957, 35707297171120604]","[0, 0]"
25,12393648,216,0xc5792b810648a237cba1bcd9b0daa64f1a8d90526f7d5517756b3a08d8b57f5d,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0xcB39E7d7B49EC8518C6832e6063623B72bf8177c,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[24528380062059822971, 112225884240301866]","[0, 0]"
55,12410098,262,0x7335c7055258463de102a8bb1b8f5b60ebf1b5d0c1fc3c7310482633ed436ce4,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x794846f3291E55E00662d37Ef048Aa716dF9ECbf,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[645508696793848095, 2497006116040891]","[0, 0]"
59,12413398,176,0x4709e68ead2b2b98ef845801f3cc1340876da0b7e2548aa4c3062f28c7d2ba42,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x873e0306bC92C6AD6e5CAD3e67A6bd4c5e8bB7FA,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[272721684075245414049, 1054962878881773324]","[0, 0]"
60,12413553,202,0x4e42aaf8c157ec5fd5b4223ba5afc840d76c6900d27a7e55a37f9ede428f6f7c,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0xFE73B5a595405baC396C329C674571a7a3DB528c,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[718655811005076849176, 2845945338563525740]","[0, 0]"
...,...,...,...,...,...,...,...,...,...
168716,20000480,113,0x50a0815ce0692e09f0327d38a24c9453ace85efaec6ff5676122fb2195c227e7,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x6a50353E2B7F2f96A7C5680c51B9f32565B04195,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, -246518217023353713]","[0, 0]"
168720,20000911,153,0xd457bb35d9a0fcb8d663e122aa34978e58698dcca95b7f735541f968aefabd45,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x6a50353E2B7F2f96A7C5680c51B9f32565B04195,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[0, -305536595445526324]","[20865377664807415550, 0]"
168722,20000937,1072,0xe36c3846113a4caa8870c2b8230d4ee992519f082dc11a0f4f2da74d3115f0f0,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x68655AD9852a99C87C0934c7290BB62CFa5D4123,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[41794868768919629908, 0]","[0, 0]"
168744,20002390,363,0xa920cc255a8d78e0231092da98f015581b9a67a7878c9959712fcb8d3b6a7a79,PoolBalanceChanged,5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014,0x64b6eBE0A55244f09dFb1e46Fe59b74Ab94F8BE1,"['0xba100000625a3754423978a60c9317c58a424e3D', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']","[3888820915608166535907, 0]","[9394595535559153097, 0]"
